In [17]:
# Self contained demo that can be run directly in colab

# Run below as colab doesn't have transformers 
# !pip install transformers

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import numpy as np
import itertools
import hashlib

In [18]:
class IdentityDetector():
    def __init__(self,public_key,allowed_distance,token_standard,model,tokenizer,device="cuda",digest_size=8):
        self.e=public_key[0] # public key exponent
        self.n=public_key[1] # public key modulus
        self.r=public_key[2] # public key 3rd prime
        self.allowed_distance=allowed_distance
        self.token_standard=token_standard
        self.device=device
        self.digest_size=digest_size
        self.tokenizer=tokenizer
        # Get vocab_size to ensure match between logit shape and vocab_size in decoder - tokenizer vocab_size does not always match the logit shape exactly.
        self.vocab_size=[x for x in model.children()][-1].out_features

    def detect(self,text):
        # Detection function will take in text, analyze it chunk by chunk to see if the chunk hashes are similar to the expected encryption (by decrypting similar hashes)

        # Tokenize the input text
        token_ids=self.tokenizer.encode(
            text, 
            add_special_tokens=True,
            return_tensors="pt")
        # Add a padding token as the generated paraphrase always starts with it
        token_ids=torch.cat((torch.tensor([[self.tokenizer.pad_token_id]]).to(self.device),token_ids),dim=-1).to(self.device)
        # Split into token_standard length chunks, retaining the end bit (i.e. the last token_ids.shape[-1] mod(token_standard) tokens) separately
        chunk_count=token_ids.shape[-1] // self.token_standard
        chunks=token_ids[0][:chunk_count*self.token_standard].reshape(chunk_count,-1)
        remainder=token_ids[0][chunk_count*self.token_standard:]
    
        # Set the seen_Sequence to be the first chunk
        seen_sequence=chunks[0]

        # List holder for whether chunks pass detection or not
        chunk_bools=[]
        # For each chunk except the first
        for chunk in chunks[1:]:

            # Hash tokens generated with public key modulus appended to front
            sequence_to_hash=[bytes(str(self.n),"utf-8")]+[bytes(str(x.item()),"utf-8") for x in seen_sequence]
            h=hashlib.shake_256(b"".join(sequence_to_hash))

            # Get a digest of a certain byte size and convert to decimal
            digest=int(h.hexdigest(self.digest_size),16)

            # Get digest mod r
            digest= digest % self.r

            # Set up the group ids
            chunk_group_ids=[]
            # For each token in the chunk
            for token in chunk:
                # Seed numpy RNG with the current hash - we get maximum digest that doesn't overflow numpy manual seeding
                # Note this replicates exactly the encoder - given a token sequence and a public key modulus, we get exactly the same seed for numpy 
                sequence_to_hash=[bytes(str(x.item()),"utf-8") for x in seen_sequence]+[bytes(str(self.n),"utf-8")]
                h=hashlib.shake_256(b"".join(sequence_to_hash))
                numpy_seed=int(h.hexdigest(4),16)
                np.random.seed(numpy_seed)

                # Get token groups from the seeded RNG
                token_group=torch.from_numpy(np.random.choice(self.vocab_size,self.vocab_size//2,replace=False)).to(torch.int64).to(self.device)

                # Add binary character to our chunk_group_ids depending on whether the token is in the token group or not
                chunk_group_ids.append(0 if token in token_group else 1)

                # Add token to seen sequence
                seen_sequence=torch.cat((seen_sequence,torch.tensor([token]).to(self.device)),dim=-1).to(self.device)

            # Put the groups list into a list of bools
            chunk_group_ids=[bool(int(x)) for x in chunk_group_ids]


            # Now we have a full chunk_group_ids which represents a binary hash. This should be close to ideal hash generated by the private key when encoding (within allowed_distance)
            # What we do next is check across similar hashes to the chunk_group_ids
            outer_bool=False
            for i in range(self.allowed_distance+1):

                # This gets all possible combinations of ways we can index into a list i times
                for x in itertools.combinations(range(self.token_standard),i):
                    # For each one, copy the list, then change the values at each index
                    copy_of_ids=chunk_group_ids.copy()

                    for j in x:
                        copy_of_ids[j]= not copy_of_ids[j]
                    # Convert copy_of_ids into a binary string, then send it to a decimal
                    encrypted_hash=int("".join([str(int(x)) for x in copy_of_ids]),2)

                    # Mod the hash with the public key 3rd prime
                    encrypted_hash=encrypted_hash % self.r

                    # Check this against the digest
                    ret_bool=self.check_hash(encrypted_hash,digest)
                    # If true, this chunk passes, so add that value to chunk bool and break both loops
                    if ret_bool:
                        chunk_bools.append(True)
                        outer_bool=True
                        break
                if outer_bool:
                    break

            # If we didn't find a pass, add false to chunk_bools
            if not outer_bool:
                chunk_bools.append(False)


            
        
        # Check if all chunks pass - all or all but one chunks must pass for sequence to pass detection
        chunk_bools=np.array(chunk_bools,dtype=bool)
        pass_bool=np.count_nonzero(chunk_bools==False)<2
        return pass_bool,chunk_bools
    
    def check_hash(self,encrypted_hash,target_hash):

        # Exponentiate the encrypted hash by the public key exponent, mod public key modulus, then take whole thing mod public key 3rd prime
        decrypt=pow(encrypted_hash,self.e,self.n) % self.r
        # If it matches the target_hash, return True, else return False
        if decrypt==target_hash:
            return True
        else:
            return False



In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [20]:
# params
token_standard=8 # length of token sequence chunks that we encrypt
arb_size=32 # Bit Size of secret primes we create in the multi prime RSA implementation
allowed_distance=3 # how much we allow generated chunks to differ from ideal chunks in terms of token group



In [21]:
public_key=(65537, 2113694530831295040457, 239) # generate public and private key


In [22]:
# Load models from Hugging Face
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)


In [23]:
# Build encoder and decoder using the keys and params
D=IdentityDetector(public_key,allowed_distance,token_standard,model,tokenizer,device)

In [24]:
# Signed prompt
signed_prompt='The world is constantly changing due to technological advancements, which include the creation of powerful language models and advanced robotics technologies. A Computer Science degree can help one be involved in these changes and apply their knowledge to everyday life, as practical applications of technology.'


In [25]:
# Does the encoded prompt pass? This should work >90% of the time.
val,truth_vals= D.detect(signed_prompt) # val is overall pass/fail, truth_vals is pass/fail for each chunk of token_standard length in the prompt
if val:
    print("This prompt was signed by the user")
else:
    print("This prompt was not signed by the user")

print(f"This prompt passed detection for {np.count_nonzero(truth_vals)} / {truth_vals.size} chunk tests")

This prompt was signed by the user
This prompt passed detection for 5 / 5 chunk tests
